<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/%EC%88%98%EC%97%85/Autogluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Autohluon은 한 모델에 대해 데이터 전처리부터 하이퍼파라미터까지 튜닝해준다.
### 스태킹기법으로 여러 모델을 사용하는 방법으로 이길 수 있다

In [ ]:
# https://www.kaggle.com/datasets/parisrohan/credit-score-classification

In [ ]:
!pip install autogluon

In [7]:
import zipfile
import os

# ZIP 파일 열기
with zipfile.ZipFile("/content/train.csv.zip", "r") as zip_ref:
    # 파일 목록 가져오기
    file_list = zip_ref.namelist()

    # 파일 목록 출력
    for file in file_list:
        print(file)

        # 파일 압축해제
        zip_ref.extract(file)

train.csv


In [8]:
import pandas as pd

df=pd.read_csv("train.csv", encoding = "utf-8")
display(df.head(10)) # print와 같음 (display)
display(df.describe())
display(df.info())

<ipython-input-8-3b6864b9c263>:3: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("train.csv", encoding = "utf-8")


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good
5,0x1607,CUS_0xd40,June,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,27.262259,22 Years and 6 Months,No,49.574949,62.430172331195294,!@9#%8,340.4792117872438,Good
6,0x1608,CUS_0xd40,July,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,22.537593,22 Years and 7 Months,No,49.574949,178.3440674122349,Low_spent_Small_value_payments,244.5653167062043,Good
7,0x1609,CUS_0xd40,August,NaN,23,#F%$D@*&8,Scientist,19114.12,1824.843333,3,...,Good,809.98,23.933795,NaN,No,49.574949,24.785216509052056,High_spent_Medium_value_payments,358.12416760938714,Standard
8,0x160e,CUS_0x21b1,January,Rick Rothackerj,28_,004-07-5839,_______,34847.84,3037.986667,2,...,Good,605.03,24.464031,26 Years and 7 Months,No,18.816215,104.291825168246,Low_spent_Small_value_payments,470.69062692529184,Standard
9,0x160f,CUS_0x21b1,February,Rick Rothackerj,28,004-07-5839,Teacher,34847.84,3037.986667,2,...,Good,605.03,38.550848,26 Years and 8 Months,No,18.816215,40.39123782853101,High_spent_Large_value_payments,484.5912142650067,Good


,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Delay_from_due_date,Num_Credit_Inquiries,Credit_Utilization_Ratio,Total_EMI_per_month
count,84998.000000,100000.000000,100000.00000,100000.000000,100000.000000,98035.000000,100000.000000,100000.000000
mean,4194.170850,17.091280,22.47443,72.466040,21.068780,27.754251,32.285173,1403.118217
std,3183.686167,117.404834,129.05741,466.422621,14.860104,193.177339,5.116875,8306.041270
min,303.645417,-1.000000,0.00000,1.000000,-5.000000,0.000000,20.000000,0.000000
25%,1625.568229,3.000000,4.00000,8.000000,10.000000,3.000000,28.052567,30.306660
50%,3093.745000,6.000000,5.00000,13.000000,18.000000,6.000000,32.305784,69.249473
75%,5957.448333,7.000000,7.00000,20.000000,28.000000,9.000000,36.496663,161.224249
max,15204.633333,1798.000000,1499.00000,5797.000000,67.000000,2597.000000,50.000000,82331.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

None

In [9]:
df.select_dtypes(include=object).columns # object type 컬럼만 가져와라

Index(['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation',
       'Annual_Income', 'Num_of_Loan', 'Type_of_Loan',
       'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Credit_Mix',
       'Outstanding_Debt', 'Credit_History_Age', 'Payment_of_Min_Amount',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score'],
      dtype='object')

In [10]:
# # 각 컬럼들의 각각의 값이 몇개인지 출력하는 코드 (딱히 필요없음)
# for i in list(df.select_dtypes(include=object).columns):
#     print(df[[f'{i}']].value_counts())
#     print("=========================")

ID     
0x10002    1
0x3c98     1
0x3cba     1
0x3cb5     1
0x3cb4     1
          ..
0x1bd73    1
0x1bd72    1
0x1bd71    1
0x1bd70    1
0xfffd     1
Length: 100000, dtype: int64
Customer_ID
CUS_0x1000     8
CUS_0x8cfe     8
CUS_0x8cbe     8
CUS_0x8cc1     8
CUS_0x8cc5     8
              ..
CUS_0x4f3b     8
CUS_0x4f3e     8
CUS_0x4f41     8
CUS_0x4f43     8
CUS_0xffd      8
Length: 12500, dtype: int64
Month   
April       12500
August      12500
February    12500
January     12500
July        12500
June        12500
March       12500
May         12500
dtype: int64
Name              
Langep                44
Stevex                44
Jessicad              39
Vaughanl              39
Deepa Seetharamanm    38
                      ..
Horowitzg              4
Alina Selyukhg         4
Ronaldf                4
Matt Scuffhamk         4
Timothyl               3
Length: 10139, dtype: int64
Age 
38      2833
28      2829
31      2806
26      2792
32      2749
        ... 
3791       1
3790     

In [13]:
import pandas as pd
# Tabular Data를 다루기 때문에 아래 라이브러리를 호출
from autogluon.tabular import TabularDataset, TabularPredictor

In [14]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

<ipython-input-14-548c513cebf5>:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('./train.csv')


In [15]:
# autogluon 학습을 위한 데이터 형태로 변환
train = TabularDataset(train_df.drop(['ID'], axis=1))
test = TabularDataset(test_df.drop(['ID'], axis=1))

In [ ]:
#학습
predictor = TabularPredictor(label='Credit_Score', eval_metric='f1_macro',).fit(train)

No path specified. Models will be saved in: "AutogluonModels/ag-20231024_005833/"
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231024_005833/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Wed Aug 30 11:19:59 UTC 2023
Disk Space Avail:   79.78 GB / 115.66 GB (69.0%)
Train Data Rows:    100000
Train Data Columns: 26
Label Column: Credit_Score
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	3 unique label values:  ['Good', 'Standard', 'Poor']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify prob

[1000]	valid_set's multi_logloss: 0.497191	valid_set's f1_macro: 0.797195


	0.8082	 = Validation score   (f1_macro)
	155.47s	 = Training   runtime
	2.27s	 = Validation runtime
Fitting model: LightGBM ...
	0.8186	 = Validation score   (f1_macro)
	204.18s	 = Training   runtime
	1.72s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7985	 = Validation score   (f1_macro)
	124.83s	 = Training   runtime
	0.26s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.7977	 = Validation score   (f1_macro)
	138.86s	 = Training   runtime
	0.36s	 = Validation runtime
Fitting model: CatBoost ...


In [ ]:
# 각각의 모델의 훈련 성능을 평가할 수 있음
ld_board = predictor.leaderboard(train, silent=True)

ld_board

In [ ]:
# 예측하기, 대회에서는 이 데이터로 제출하면 됨
pred_y = predictor.predict(test)

In [ ]:
# 테스트 데이터(정답값 있는 경우 사용), kaggle 같은 대회에서는 test 정답값 없음
from sklearn.metrics import f1_score, accuracy_score

f1_score(pred_y.test(['Credit_score']))
accuracy_score(pred_y.test('Credit_score'))